# self-attention

这里仅仅尝试复现论文中的模型，从最简单的 attention开始，到之后的 multi-head attention, 再到 block, 再到整个模型。

## 1. scaled-dot-product-attention

![scaled-dot-product-attention](./img/scaled-dot-product-attention.png)

$$Attention(\boldsymbol{Q},\boldsymbol{K},\boldsymbol{V}) = softmax\left(\frac{\boldsymbol{Q}\boldsymbol{K}^{\top}}{\sqrt{d_k}}\right)\boldsymbol{V}$$

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import *
import numpy as np
from torch.nn.parameter import Parameter

# No Mask
class ScaledDotProductAttention(nn.Module):

    def __init__(self, dk):
        '''scaled-dot-product-attention
            parameters: dk (dk)
        '''
        super(ScaledDotProductAttention, self).__init__()
        self.dk = dk

    def forward(self, Q, K, V):
        ''' forward step
            parameters: Q (n*dk), K(m*dk), V(n*dv)
        '''
        qk = torch.mm(Q, torch.t(K)) # (n*dk) x (dk*m) -> n*m
        vec = qk / torch.sqrt(self.dk) # n*m -> n*m
        weight = F.softmax(vec, dim=1) # n*m -> n*1
        attention_V = torch.matmul(weight, V) # (n*1) x (n*dv) -> n*dv

In [ ]:
# test data
Q1 = torch.randn()
V1 = 
k1 = 